# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [22]:
using Pkg
pkg"add NLPModels#master"

    Cloning git-repo `https://github.com/JuliaSmoothOptimizers/NLPModels.jl.git`


ching: [========================================>]  99.9 %[==>                                      ]  3.9 %Fetching: [===>                                     ]  5.9 %.8 %=====>                                   ]  10.9 %4 %Fetching: [============>                            ]  28.4 %>                           ]  31.0 %Fetching: [==============>                          ]  32.8 % [===============>                         ]  35.5 % %4 %Fetching: [==================>                      ]  43.2 %Fetching: [===================>                     ]  45.8 %Fetching: [====================>                    ]  48.4 %]  49.9 % [=====================>                   ]  52.1 %Fetching: [========================>                ]  59.1 %Fetching: [=========================>               ]  61.5 %>              ]  64.0 % [=============================>           ]  70.5 % [==============================>          ]  72.5 % [==============================>          ]  74.1 %>         ]  

   Updating git-repo `https://github.com/JuliaSmoothOptimizers/NLPModels.jl.git`


Fetching: [========================================>]  100.0 %===>                                     ]  5.8 %]  9.9 %Fetching: [======>                                  ]  13.2 %                               ]  20.3 %]  28.7 %38.1 % [===================>                     ]  45.5 %]  48.3 %Fetching: [=====================>                   ]  51.0 %55.4 % [========================>                ]  59.5 %               ]  62.4 % [============================>            ]  68.3 %71.2 %==============================>          ]  75.0 % [================================>        ]  78.6 %3 %86.1 %Fetching: [====================================>    ]  89.3 %]  91.8 % ]  96.7 %]  99.4 %

   Updating registry at `C:\Users\Geoffroy Leconte\.julia\registries\General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %.0 % [====>                                    ]  8.7 %Fetching: [======>                                  ]  12.9 %=======>                                 ]  15.7 % [=========>                               ]  20.1 %30.3 %>                          ]  33.8 % %.9 %                   ]  51.1 %.6 %64.4 %============================>            ]  68.3 %]  72.7 %.5 %8 % [=======================================> ]  96.3 %

  Resolving package versions...
  Installed ZeroMQ_jll ─ v4.3.2+3
   Updating `C:\Users\Geoffroy Leconte\.julia\environments\v1.4\Project.toml`
  [a4795742] ~ NLPModels v0.12.4 ⇒ v0.12.4 #master (https://github.com/JuliaSmoothOptimizers/NLPModels.jl.git)
   Updating `C:\Users\Geoffroy Leconte\.julia\environments\v1.4\Manifest.toml`
  [a4795742] ~ NLPModels v0.12.4 ⇒ v0.12.4 #master (https://github.com/JuliaSmoothOptimizers/NLPModels.jl.git)
  [8f1865be] ↑ ZeroMQ_jll v4.3.2+2 ⇒ v4.3.2+3


In [1]:

using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [2]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1.
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1.
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    return x_tilde
end

function starting_points(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(-ρ*ones(n_cols),spzeros(n_rows))

    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :U))
    init_xλ = J_fact \ [c ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = 0., 0.
    else
        δx_l1 = max(-1.5*minimum(x0_l1 - lvar[ilow]), 0.)
        δs_l1 = max(-1.5*minimum(s0_l[ilow]), 0.)
    end

    if length(iupp) == 0
        δx_u1, δs_u1 = 0., 0.
    else
        δx_u1 = max(-1.5*minimum((uvar[iupp] - x0_u1)), 0.)
        δs_u1 = max(-1.5*minimum(s0_u[iupp]), 0.)
    end
    
    x0_l1 .+= δx_l1
    x0_u1 .-= δx_u1
    s0_l1 = s0_l[ilow] .+ δs_l1
    s0_u1 = s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = s0_l1'*(x0_l1 - lvar[ilow]), s0_u1'*(uvar[iupp] - x0_u1)
    if length(ilow) == 0
        δx_l2, δs_l2 = 0., 0.
    else
        δx_l2 = δx_l1 + 0.5 *  xs_l1 / sum(s0_l1)
        δs_l2 = δs_l1 + 0.5 * xs_l1 / sum(x0_l1-lvar[ilow])
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = 0., 0.
    else
        δx_u2 = δx_u1 + 0.5 *  xs_u1 / sum(s0_u1)
        δs_u2 = δs_u1 + 0.5 * xs_u1 / sum(uvar[iupp]-x0_u1)
    end
    println(δx_l1)
    x0[ilow] .+= δx_l2 
    x0[iupp] .-= δx_u2
    s0_l[ilow] = s0_l[ilow] .+ max(δs_l2, δs_u2)
    s0_u[iupp] = s0_u[iupp] .+ max(δs_l2, δs_u2)

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end



function starting_points2(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(-ones(n_cols),spzeros(n_rows))

    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :U))
    init_xλ = J_fact \ [zeros(n_cols) ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]

    for i in ilow
        δx_l = max(-1.5*(x0[i] - lvar[i]), 1.)
        δs_l = max(-1.5*s0_l[i], 1e-6)
        x0[i] += δx_l 
        s0_l[i] += δs_l 
    end

    for i in iupp
        δx_u = max(-1.5*(uvar[i] - x0[i]), 1.)
        δs_u = max(-1.5*s0_u[i], 1.e-6)
        x0[i] -= δx_u 
        s0_u[i] += δs_u 
    end

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end


function starting_points3(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(ρ*ones(n_cols),spzeros(n_rows))

    #J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :U))
    J_fact = ldl(Symmetric(J_augm-Diagonal(tmp_diag), :U))
    J_P = J_fact.P
    init_xλ = J_fact \ [zeros(n_cols) ; b]
    #init_xλ2 = J_fact \ [c ; zeros(n_rows)]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = @views dual_val[ilow]
    s0_u[iupp] = @views -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = 0., 0.
    else
        δx_l1 = @views max(-1.5*minimum(x0_l1 - lvar[ilow]), 1.e0)
        δs_l1 = @views max(-1.5*minimum(s0_l[ilow]), 1.e-4)
    end
    
    if length(iupp) == 0
        δx_u1, δs_u1 = 0., 0.
    else
        δx_u1 = @views max(-1.5*minimum((uvar[iupp] - x0_u1)), 1.e0)
        δs_u1 = @views max(-1.5*minimum(s0_u[iupp]), 1.e-4)
    end
    
    x0_l1 .+= δx_l1
    x0_u1 .-= δx_u1
    s0_l1 = @views s0_l[ilow] .+ δs_l1
    s0_u1 = @views s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = @views s0_l1'*(x0_l1 - lvar[ilow]), s0_u1'*(uvar[iupp] - x0_u1)
    if length(ilow) == 0
        δx_l2, δs_l2 = 0., 0.
    else
        δx_l2 = δx_l1 + 0.5 *  xs_l1 / sum(s0_l1)
        δs_l2 = @views δs_l1 + 0.5 * xs_l1 / sum(x0_l1-lvar[ilow])
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = 0., 0.
    else
        δx_u2 = δx_u1 + 0.5 *  xs_u1 / sum(s0_u1)
        δs_u2 = @views δs_u1 + 0.5 * xs_u1 / sum(uvar[iupp]-x0_u1)
    end
    δx = max(δx_l2, δx_u2)
    δs = max(δs_l2, δs_u2)
    x0[ilow] .+= δx
    x0[iupp] .-= δx
    s0_l[ilow] = @views s0_l[ilow] .+ δs
    s0_u[iupp] = @views s0_u[iupp] .+ δs

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert @views all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u, J_P
end

function starting_points4(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    x0 = A\b
    
    if length(ilow) == 0
        δx_l1 = 0.
    else
        δx_l1 = max(-2.5*minimum(x0[ilow] - lvar[ilow]), 1.)
    end

    if length(iupp) == 0
        δx_u1 = 0.
    else
        δx_u1 = max(-2.5*minimum((uvar[iupp] - x0[iupp])), 1.)
    end
    
    x0[ilow] .+= δx_l1
    x0[iupp] .-= δx_u1
    
    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end
    
    @assert all(x0 .> lvar) && all(x0 .< uvar)
    λ0 = sparse(A') \ (c+Q*x0)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = dual_val[iupp]
    
    s0_l[ilow] .= 1.
    s0_u[iupp] .= 1.    
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u
end

starting_points4 (generic function with 1 method)

In [3]:
function compute_α_dual(v, dir_v)
    n = length(v)
    if n == 0
        return 1.
    end
    α = 1.
    for i=1:n
        if dir_v[i] < 0.
            α_new = -v[i] * 0.999 / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    α_l, α_u = 1., 1.
    for i=1:n
        if dir_v[i] > 0.
            α_u_new = (uvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < 0.
            α_l_new = (lvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end



function compute_μ(x_minus_lvar, uvar_minus_x, s_l, s_u, nb_low, nb_upp)
    return (s_l' * x_minus_lvar + s_u' * uvar_minus_x) / (nb_low + nb_upp)
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [4]:
function solve_augmented_system_aff!(J_fact, Δ_aff, Δ_x_λ, rc, rb, x_minus_lvar, uvar_minus_x, 
                                     s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)
    
    F_x_λ_aff = [-rc 
                 -rb]
    F_x_λ_aff[ilow] += @views s_l[ilow]
    F_x_λ_aff[iupp] -= @views s_u[iupp]
    
    Δ_x_λ = ldiv!(Δ_x_λ , J_fact, F_x_λ_aff)
#     Δ_aff = @views [Δ_x_λ
#                     -s_l[ilow] - s_l[ilow].*Δ_x_λ[1:n_cols][ilow]./x_minus_lvar
#                     -s_u[iupp] + s_u[iupp].*Δ_x_λ[1:n_cols][iupp]./uvar_minus_x]
    Δ_aff[1:n_cols+n_rows] = Δ_x_λ
    Δ_aff[n_cols+n_rows+1:n_cols+n_rows+n_low] = @views -s_l[ilow] - s_l[ilow].*Δ_x_λ[1:n_cols][ilow]./x_minus_lvar
    Δ_aff[n_cols+n_rows+n_low+1:end] = @views -s_u[iupp] + s_u[iupp].*Δ_x_λ[1:n_cols][iupp]./uvar_minus_x
    return Δ_aff
end

function solve_augmented_system_cc!(J_fact, Δ_cc, Δ_x_λ ,Δ_aff, σ, μ,x_minus_lvar, uvar_minus_x, 
                                    s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)


    rxs_l = @views -σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
    rxs_u = @views σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]

    F_x_λ_cc = zeros(n_cols+n_rows)
    F_x_λ_cc[ilow] += rxs_l./x_minus_lvar
    F_x_λ_cc[iupp] += rxs_u./uvar_minus_x

#     F_x_λ_cc = [ I_low*inv_X_L*rxs_l + I_op_upp*inv_X_U*rxs_u
#                                    Or                 ]
    
    Δ_x_λ = ldiv!(Δ_x_λ , J_fact, F_x_λ_cc)

#     Δ_cc = @views [Δ_x_λ
#                    -(rxs_l+s_l[ilow].*Δ_x_λ[1:n_cols][ilow])./x_minus_lvar
#                    (rxs_u+s_u[iupp].*Δ_x_λ[1:n_cols][iupp])./uvar_minus_x ]
    Δ_cc[1:n_cols+n_rows] = Δ_x_λ
    Δ_cc[n_cols+n_rows+1:n_cols+n_rows+n_low] = @views -(rxs_l+s_l[ilow].*Δ_x_λ[1:n_cols][ilow])./x_minus_lvar
    Δ_cc[n_cols+n_rows+n_low+1:end] = @views (rxs_u+s_u[iupp].*Δ_x_λ[1:n_cols][iupp])./uvar_minus_x 
    return Δ_cc
end

solve_augmented_system_cc! (generic function with 1 method)

In [5]:
function check_frontier!(γf, x, x_minus_lvar, uvar_minus_x, s_l, s_u, Δ, α_pri, α_dual, 
                        lvar, uvar, ilow, iupp, n_low, n_upp, n_rows, n_cols)
    
    x_minus_lvar_sup = @views a_plus_αb(x_minus_lvar, α_pri, Δ[1:n_cols][ilow], n_low)
    uvar_minus_x_sup = @views a_plus_αb(uvar_minus_x, -α_pri, Δ[1:n_cols][iupp], n_upp)
    s_l_sup = @views a_plus_αb(s_l[ilow], α_dual, Δ[n_rows+n_cols+1: n_rows+n_cols+n_low], n_low)
    s_u_sup = @views a_plus_αb(s_u[iupp], α_dual,  Δ[n_rows+n_cols+n_low+1: end], n_upp)
    μ_sup = compute_μ(x_minus_lvar_sup, uvar_minus_x_sup, s_l_sup, s_u_sup, n_low, n_upp)
    f_pri, f_pri_tmp = 0., 0.
    f_dual, f_dual_tmp = 0., 0.
    i_l, i_u = 0, 0
    ilow_pad, iupp_pad = zeros(Int, n_cols), zeros(Int, n_cols)
    ilow_pad[ilow] = 1:n_low
    iupp_pad[iupp] = 1:n_upp
    for i=1:n_cols 
        if ilow_pad[i] != 0 && iupp_pad[i] != 0
            i_l, i_u = ilow_pad[i], iupp_pad[i]
            if (x_minus_lvar_sup[i_l] == 0. || uvar_minus_x_sup[i_u] == 0.)
                f_pri_tmp = (2.0*γf*μ_sup - 
                            x_minus_lvar[i_l]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]) -
                            uvar_minus_x[i_u]*(s_u_sup[i_u]+α_dual*Δ[n_rows+n_cols+n_low+i_u])) / 
                                (α_pri * Δ[i]*
                                (s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]-s_u_sup[i_u]-α_dual*Δ[n_rows+n_cols+n_low+i_u]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if (s_l_sup[i_l] == 0. || uvar_minus_x_sup[i_u] == 0.)
                f_dual_tmp = (2.0*γf*μ_sup - s_l[i_l]*x_minus_lvar_sup[i_l] - s_u[i_u]*uvar_minus_x_sup[i_u]) /
                                (α_dual * (Δ[n_rows+n_cols+i_l]*x_minus_lvar_sup[i_l] +
                                     Δ[n_rows+n_cols+n_low+i_u]*uvar_minus_x_sup[i_u]))
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end  
            end
        elseif ilow_pad[i] != 0 && iupp_pad[i] == 0
            i_l = ilow_pad[i]
            if x_minus_lvar_sup[i_l] == 0. 
                f_pri_tmp = (γf*μ_sup - x_minus_lvar[i_l]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l])) / 
                                (α_pri * Δ[i]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if s_l_sup[i_l] == 0.
                f_dual_tmp = (γf*μ_sup - s_l[i_l]*x_minus_lvar_sup[i_l] ) /
                                (α_dual * Δ[n_rows+n_cols+i_l]*x_minus_lvar_sup[i_l])
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end
            end 
        elseif ilow_pad[i] == 0 && iupp_pad[i] != 0
            i_u = iupp_pad[i]
            if uvar_minus_x_sup[i_u] == 0.
                f_pri_tmp = (γf*μ_sup - uvar_minus_x[i_u]*(s_u_sup[i_u]+α_dual*Δ[n_rows+n_cols+n_low+i_u])) / 
                                (α_pri * Δ[i]*(-s_u_sup[i_u]-α_dual*Δ[n_rows+n_cols+n_low+i_u]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if s_u[i_u] == 0.
                f_dual_tmp = (γf*μ_sup - s_u[i_u]*uvar_minus_x_sup[i_u]) /
                            (α_dual * Δ[n_rows+n_cols+n_low+i_u]*uvar_minus_x_sup[i_u])
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end  
            end
        end
    end
    
    α_pri *= max(1.0-γf, f_pri)
    α_dual *= max(1.0-γf, f_dual)
    
    return α_pri, α_dual
end  

function check_frontier2!(γf, x, x_minus_lvar, uvar_minus_x, s_l, s_u, Δ, α_pri, α_dual, 
                        lvar, uvar, ilow, iupp, n_low, n_upp, n_rows, n_cols)
    
    x_minus_lvar_sup = a_plus_αb(x_minus_lvar, α_pri, Δ[1:n_cols][ilow], n_low)
    uvar_minus_x_sup = a_plus_αb(uvar_minus_x, -α_pri, Δ[1:n_cols][iupp], n_upp)
    s_l_sup = a_plus_αb(s_l[ilow], α_dual, Δ[n_rows+n_cols+1: n_rows+n_cols+n_low], n_low)
    s_u_sup = a_plus_αb(s_u[iupp], α_dual,  Δ[n_rows+n_cols+n_low+1: end], n_upp)
    μ_sup = compute_μ(x_minus_lvar_sup, uvar_minus_x_sup, s_l_sup, s_u_sup, n_low, n_upp)
    f_pri_l, f_pri_l_tmp = 0., 0.
    f_pri_u, f_pri_u_tmp = 0., 0.
    f_dual_l, f_dual_l_tmp = 0., 0.
    f_dual_u, f_dual_u_tmp = 0., 0.
    i_l, i_u = 0, 0
    ilow_pad, iupp_pad = zeros(Int, n_cols), zeros(Int, n_cols)
    ilow_pad[ilow] = 1:n_low
    iupp_pad[iupp] = 1:n_upp
    for i=1:n_cols 
        if ilow_pad[i] != 0 
            i_l = ilow_pad[i]
            if x_minus_lvar_sup[i_l] == 0. 
                f_pri_l_tmp = (γf*μ_sup - x_minus_lvar[i_l]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l])) / 
                                (α_pri * Δ[i]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]))      
                if f_pri_l_tmp > f_pri_l
                    f_pri_l = f_pri_l_tmp
                end
            end
            if s_l_sup[i_l] == 0.
                f_dual_l_tmp = (γf*μ_sup - s_l[i_l]*x_minus_lvar_sup[i_l] ) /
                                (α_dual * Δ[n_rows+n_cols+i_l]*x_minus_lvar_sup[i_l])
                if f_dual_l_tmp > f_dual_l
                    f_dual_l = f_dual_l_tmp
                end
            end 
        end
        if iupp_pad[i] != 0
            i_u = iupp_pad[i]
            if uvar_minus_x_sup[i_u] == 0.
                f_pri_u_tmp = (γf*μ_sup - uvar_minus_x[i_u]*(s_u_sup[i_u]+α_dual*Δ[n_rows+n_cols+n_low+i_u])) / 
                                (α_pri * Δ[i]*(-s_u_sup[i_u]-α_dual*Δ[n_rows+n_cols+n_low+i_u]))      
                if f_pri_u_tmp > f_pri_u
                    f_pri_u = f_pri_u_tmp
                end
            end
            if s_u[i_u] == 0.
                f_dual_u_tmp = (γf*μ_sup - s_u[i_u]*uvar_minus_x_sup[i_u]) /
                            (α_dual * Δ[n_rows+n_cols+n_low+i_u]*uvar_minus_x_sup[i_u])
                if f_dual_u_tmp > f_dual_u
                    f_dual_u = f_dual_u_tmp
                end  
            end
        end
    end
    f_pri = min(f_pri_l, f_pri_u)
    f_dual = min(f_dual_l, f_dual_u)
    α_pri *= max(1.0-γf, f_pri)
    α_dual *= max(1.0-γf, f_dual)
    
    return α_pri, α_dual
end

check_frontier2! (generic function with 1 method)

In [6]:
function a_plus_αb(a, α, b, n)
    # a, b same length, α float
    result = zeros(n)
    for i=1:n
        result[i] = a[i] + α * b[i]
    end
    return result
end

function a_plus_equal_αb(a, α, b, n)
    # a, b same length, α float
    for i=1:n
        a[i] +=  α * b[i]
    end
    return a
end
   

function equilibrationRT(A, n_rows, n_cols)
    r = zeros(n_rows)
    t = zeros(n_cols)
    AT = sparse(A')
    for j=1:n_rows
        i = AT.colptr[j]
        k = AT.colptr[j+1] - 1
        r_j_inv = i <= k ? norm(AT.nzval[i:k], Inf) : 0.0
        if r_j_inv > 0.0
          r[j] = 1/r_j_inv
        end

    end
    
    for j=1:n_cols
        i = A.colptr[j]
        k = A.colptr[j+1] - 1
        t_j_inv = i <= k ? norm(A.nzval[i:k], Inf) : 0.0
        if t_j_inv > 0.0
          t[j] = 1/t_j_inv
        end
    end
    return Diagonal(r), Diagonal(t)
end

function get_norm_cols(A, n_cols)
    c = ones(n_cols)
    for j=1:n_cols
        i = A.colptr[j]
        k = A.colptr[j+1] - 1
        c[j] = i <= k ? sqrt(norm(A.nzval[i:k], Inf)) : 1.0
    end
    return c
end
    

function scaling_Ruiz!(A, n_rows, n_cols, ϵ; max_iter = 100)
    AT = sparse(A')
    D1, D2 = I(n_rows), I(n_cols)
    r, c = get_norm_cols(AT, n_rows), get_norm_cols(A, n_cols)
    convergence = maximum(abs.(1.0 .- r)) <= ϵ && maximum(abs.(1.0 .- c)) <= ϵ
    inv_Dr, inv_Dc = inv(Diagonal(r)), inv(Diagonal(c))
    A = rmul!(A, inv_Dc)
    A = lmul!(inv_Dr, A)
    AT = lmul!(inv_Dc, AT)
    AT = rmul!(AT, inv_Dr)
    D1 *= inv_Dr
    D2 *= inv_Dc
    k = 1
    while !convergence && k < max_iter
        r, c = get_norm_cols(AT, n_rows), get_norm_cols(A, n_cols)
        convergence = maximum(abs.(1.0 .- r)) <= ϵ && maximum(abs.(1.0 .- c)) <= ϵ
        inv_Dr, inv_Dc = inv(Diagonal(r)), inv(Diagonal(c))
        A = rmul!(A, inv_Dc)
        A = lmul!(inv_Dr, A)
        AT = lmul!(inv_Dc, AT)
        AT = rmul!(AT, inv_Dr)
        D1 *= inv_Dr
        D2 *= inv_Dc
        k += 1
    end
    return A, D1, D2, inv(D1), inv(D2)
end

scaling_Ruiz! (generic function with 1 method)

In [7]:
function mehrotraPCQuadBounds(QM; max_iter=100, ϵ_pdd=1e-8, ϵ_rb=1e-6, ϵ_rc=1e-6,
                              tol_Δx=1e-16, ϵ_μ=0., max_time=60., scaling=true,
                              display=true)

    start_time = time()
    elapsed_time = 0.0

    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp = [QM.meta.ilow; QM.meta.irng], [QM.meta.iupp; QM.meta.irng] # finite bounds index
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    c = grad(QM, Oc)
    A = jac(QM, Oc)
    n_rows, n_cols = size(A)
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    Q = hess(QM, Oc)  # lower triangular

    if scaling
#         R, T = equilibrationRT(A, n_rows, n_cols)
#         inv_R_vec = zeros(n_rows)
#         for i=1:n_rows
#             if R[i,i] != 0.
#                 inv_R_vec[i] = 1/R[i,i]
#             end
#         end
#         inv_T_vec = zeros(n_cols)
#         for i=1:n_cols
#             if T[i,i] != 0.
#                 inv_T_vec[i] = 1/T[i,i]
#             end
#         end
#         inv_R = Diagonal(inv_R_vec)
#         inv_T = Diagonal(inv_T_vec)
#         A = R*A*T
        A, D1, D2, inv_D1, inv_D2 = scaling_Ruiz!(A, n_rows, n_cols, 1e-3)
        Q = Q*D2^2
        c = D2*c
        b = D1*b
        lvar = inv_D2*lvar
        uvar = inv_D2*uvar
#         A = rmul!(A, T)
#         A = lmul!(R, A)
#         Q = rmul!(Q, T^2)
#         c = mul!(c, T, c)
#         b = mul!(b, R, b)
#         lvar = mul!(lvar, inv_T, lvar)
#         uvar = mul!(uvar, inv_T, uvar)
    end

    Arows, Acols, Avals = findnz(A)
    c0 = obj(QM, Oc)
    Qrows, Qcols, Qvals = findnz(Q)
    Q_sym = Symmetric(Q, :L)

    # init regularization values
    ρ, δ = 1e5*sqrt(eps()), 1e6*sqrt(eps()) # 1e6, 1e-1 ok

    J_augmrows = vcat(Qcols, Acols, n_cols+1:n_cols+n_rows, 1:n_cols)
    J_augmcols = vcat(Qrows, Arows.+n_cols, n_cols+1:n_cols+n_rows, 1:n_cols)
    tmp_diag = zeros(n_cols)
    J_augmvals = vcat(-Qvals, Avals, δ*ones(n_rows), tmp_diag)
    J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)

    x, λ, s_l, s_u, J_P = @views starting_points3(Q_sym, A, b, c, lvar, uvar, ilow, iupp,
                                     QM.meta.irng, J_augm, 1.0e-7, n_rows, n_cols)

#     x = init_x0_lsq(A, b, lvar, uvar)
#     @assert all(x .> lvar) && all(x .< uvar)
#     s_l, s_u = copy(Oc), copy(Oc)
#     s_l[ilow] .= 1.
#     s_u[iupp] .= 1.

    Qx = Q_sym * x
#     λ = sparse(A') \ (c+Qx) # least square initialisation, s_0 = stilde_0
    ATλ = A' * λ
    Ax = A * x
    rb = Ax - b
    rc = -Qx + ATλ + s_l - s_u - c

    x_minus_lvar = @views x[ilow] - lvar[ilow]
    uvar_minus_x = @views uvar[iupp] - x[iupp]
    μ = @views compute_μ(x_minus_lvar, uvar_minus_x,
                         s_l[ilow], s_u[iupp],
                         n_low, n_upp)

    k = 0
    Δ_aff = zeros(n_cols+n_rows+n_low+n_upp)
    Δ_cc = zeros(n_cols+n_rows+n_low+n_upp)
    Δ = zeros(n_cols+n_rows+n_low+n_upp)
    Δ_x_λ = zeros(n_cols+n_rows)

    # stopping criterion
    xTQx_2 = x' * Qx / 2.
    cTx = c' * x
    pri_obj = xTQx_2 + cTx + c0
    dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0
    pdd = abs(pri_obj - dual_obj ) / (1. + abs(pri_obj))
    max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
    optimal = pdd < ϵ_pdd && max_rb < ϵ_rb && max_rc < ϵ_rc

    n_Δx = 0.
    small_Δx, small_μ = false, μ < ϵ_μ
    Δt = time() - start_time
    tired = Δt > max_time

    # display
    if display == true
        @info log_header([:k, :pri_obj, :pdd, :max_rb, :max_rc, :n_Δx, :μ],
                         [Int, Float64, Float64, Float64, Float64, Float64, Float64, Float64],
                        hdr_override=Dict(:k=>"Iter", :pri_obj=>"primal", :pdd=>"pdd",
                                          :max_rb=>"rb cond", :max_rc=>"rc cond",
                                          :n_Δx=>"‖Δx‖", :μ=>"μ"))
        @info log_row([k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
    end

    while k<max_iter && !optimal && !tired # && !small_μ && !small_μ

            # Affine scaling direction
        tmp_diag .= -ρ
        tmp_diag[ilow] .-= @views s_l[ilow] ./ x_minus_lvar
        tmp_diag[iupp] .-= @views s_u[iupp] ./ uvar_minus_x
        J_augmvals[end-n_cols+1:end] = tmp_diag

        J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)

        #J_fact = ldlt(Symmetric(J_augm, :U))
        #J_fact = lu(Symmetric(J_augm), check=true)
        J_fact = ldl(Symmetric(J_augm, :U), J_P)

        Δ_aff = solve_augmented_system_aff!(J_fact, Δ_aff, Δ_x_λ, rc, rb, x_minus_lvar, uvar_minus_x,
                                            s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)

        α_aff_pri = @views compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = @views compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = @views compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)

        μ_aff = @views compute_μ(a_plus_αb(x_minus_lvar, α_aff_pri, Δ_aff[1:n_cols][ilow], n_low),
                                 a_plus_αb(uvar_minus_x, -α_aff_pri, Δ_aff[1:n_cols][iupp], n_upp),
                                 a_plus_αb(s_l[ilow], α_aff_dual_final, Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low], n_low),
                                 a_plus_αb(s_u[iupp], α_aff_dual_final,  Δ_aff[n_rows+n_cols+n_low+1: end], n_upp),
                                 n_low, n_upp)

        σ = (μ_aff / μ)^3

        # corrector and centering step

        Δ_cc = solve_augmented_system_cc!(J_fact, Δ_cc, Δ_x_λ , Δ_aff, σ, μ,x_minus_lvar, uvar_minus_x,
                                          s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)


        Δ = Δ_aff + Δ_cc # final direction

        α_pri = @views compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = @views compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = @views compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        α_pri, α_dual_final = check_frontier!(1.0e-3, x, x_minus_lvar, uvar_minus_x, s_l, s_u,
                                              Δ, α_pri, α_dual_final, lvar, uvar, ilow, iupp,
                                              n_low, n_upp, n_rows, n_cols)

        # new parameters
        x = @views a_plus_equal_αb(x, α_pri, Δ[1:n_cols], n_cols)
        λ = @views a_plus_equal_αb(λ, α_dual_final,
                                   Δ[n_cols+1: n_rows+n_cols], n_rows)
        s_l[ilow] = @views a_plus_equal_αb(s_l[ilow], α_dual_final,
                                           Δ[n_rows+n_cols+1: n_rows+n_cols+n_low], n_low)
        s_u[iupp] = @views a_plus_equal_αb(s_u[iupp], α_dual_final,
                                           Δ[n_rows+n_cols+n_low+1: end], n_upp)
        n_Δx = @views α_pri * norm(Δ[1:n_cols])
        x_minus_lvar = @views x[ilow] - lvar[ilow]
        uvar_minus_x = @views uvar[iupp] - x[iupp]

        μ = @views compute_μ(x_minus_lvar, uvar_minus_x,
                             s_l[ilow], s_u[iupp],
                             n_low, n_upp)

        Qx = mul!(Qx, Q_sym, x)
        xTQx_2 =  x' * Qx / 2.
        ATλ = mul!(ATλ, A', λ)
        Ax = mul!(Ax, A, x)
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0

        rb = Ax - b
        rc = -Qx + ATλ + s_l - s_u - c

        # update stopping criterion values:

        pdd = abs(pri_obj - dual_obj ) / (1. + abs(pri_obj))
        max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
        optimal = pdd < ϵ_pdd && max_rb < ϵ_rb && max_rc < ϵ_rc
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1

        if δ >= 100*sqrt(eps())
            δ /= 10
            J_augmvals[end-n_cols-n_rows+1:end-n_cols] .= δ
        end
        if ρ >= 1*sqrt(eps())
            ρ /= 10
        end

        Δt = time() - start_time
        tired = Δt > max_time

        if display == true
            @info log_row([k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
        end

    end

    if k>= max_iter
        status = :max_iter
    elseif tired
        status = :max_time
    elseif optimal
        status = :acceptable
    else
        status = :unknown
    end

    if scaling
        x = D2*x
        Q_sym2 = Symmetric(Q*inv_D2^2)
        Qx = mul!(Qx, Q_sym2, x)
        xTQx_2 =  x' * Qx / 2.
        A = rmul!(A, inv_D2)
        A = lmul!(inv_D1, A)
        b = mul!(b, inv_D1, b)
        ATλ = mul!(ATλ, A', λ)
        Ax = mul!(Ax, A, x)
        cTx = c' *inv_D2* x
        pri_obj = xTQx_2 + cTx + c0
        lvar = D2 * lvar
        uvar = D2 * uvar
        dual_obj = b' * λ - xTQx_2 + view(s_l,ilow)'*view(lvar,ilow) -
                    view(s_u,iupp)'*view(uvar,iupp) +c0
    end
    
    elapsed_time = time() - start_time

    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj ,
                                  dual_feas = max_rc,
                                  primal_feas = max_rb,
                                  multipliers = λ,
                                  multipliers_L = s_l,
                                  multipliers_U = s_u,
                                  iter = k, elapsed_time=elapsed_time)
    return stats
end

mehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [8]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];

In [9]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [10]:
stats_mpc1 =  mehrotraPCQuadBounds(SM, 20)
println(stats_mpc1)

MethodError: MethodError: no method matching mehrotraPCQuadBounds(::QuadraticModel, ::Int64)
Closest candidates are:
  mehrotraPCQuadBounds(::Any; max_iter, ϵ_pdd, ϵ_rb, ϵ_rc, tol_Δx, ϵ_μ, max_time, scaling, display) at In[7]:5

In [11]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
stats_mpc_verif =  mehrotraPCQuadBounds(SM_verif, 20);
println(stats_mpc_verif)

MethodError: MethodError: no method matching mehrotraPCQuadBounds(::QuadraticModel, ::Int64)
Closest candidates are:
  mehrotraPCQuadBounds(::Any; max_iter, ϵ_pdd, ϵ_rb, ϵ_rc, tol_Δx, ϵ_μ, max_time, scaling, display) at In[7]:5

In [12]:
### pb verif 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
stats_mpc_verif2 =  mehrotraPCQuadBounds(SM_verif2, 20);
println(stats_mpc_verif2)

MethodError: MethodError: no method matching mehrotraPCQuadBounds(::QuadraticModel, ::Int64)
Closest candidates are:
  mehrotraPCQuadBounds(::Any; max_iter, ϵ_pdd, ϵ_rb, ϵ_rc, tol_Δx, ϵ_μ, max_time, scaling, display) at In[7]:5

# Lecture des donnees .SIF

In [13]:

function createQuadraticModel(qpdata)
    # probleme du point initial
    x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, x0=x0)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

In [14]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [15]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
SM2.meta

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: Generic-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   



In [54]:
@show SM2

SM2 = SlackModel - Model with slack variables
  Problem name: Generic-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   

  Counters:
 

SlackModel - Model with slack variables
  Problem name: Generic-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   

  Counters:
       

In [16]:

stats_mpc2 =  mehrotraPCQuadBounds(SM2, 100)
println(stats_mpc2)

MethodError: MethodError: no method matching mehrotraPCQuadBounds(::SlackModel, ::Int64)
Closest candidates are:
  mehrotraPCQuadBounds(::Any; max_iter, ϵ_pdd, ϵ_rb, ϵ_rc, tol_Δx, ϵ_μ, max_time, scaling, display) at In[7]:5

In [17]:
@benchmark mehrotraPCQuadBounds(SM2, 100, display=false)

MethodError: MethodError: no method matching mehrotraPCQuadBounds(::SlackModel, ::Int64; display=false)
Closest candidates are:
  mehrotraPCQuadBounds(::Any; max_iter, ϵ_pdd, ϵ_rb, ϵ_rc, tol_Δx, ϵ_μ, max_time, scaling, display) at In[7]:5

In [57]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
SM3 = SlackModel(QM3)
SM3.meta

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  Problem name: Generic-slack
   All variables: ████████████████████ 68     All constraints: ████████████████████ 43    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ██████████████⋅⋅⋅⋅⋅⋅ 47               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 12               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 43    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 43    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 89.30% sparsity)   313   



In [58]:
stats_mpc3 =  mehrotraPCQuadBounds(SM3, 100);
println(stats_mpc3)

Iter    primal      pdd      rb cond    rc cond    step x       μ
   0   1.17e+01   3.4e+01    6.9e+02    6.1e+00    0.0e+00   6.2e+00
   1  -2.54e+00   5.7e+00    2.6e+02    2.8e+00    4.6e+00   1.2e+00
   2  -3.99e+00   1.5e-01    9.1e+01    4.2e-01    3.9e+00   3.4e-01
   3  -7.45e+00   5.7e-01    3.7e+01    3.4e-01    2.4e+01   2.0e-01
   4  -1.65e+01   8.2e-01    1.1e+01    2.6e-01    5.6e+01   2.0e-01
   5  -1.02e+02   9.7e-01    7.7e-03    1.9e-01    4.3e+02   3.8e-01
   6  -2.25e+02   9.7e-01    7.3e-03    1.9e-01    6.2e+02   7.7e-01
   7  -3.99e+02   9.7e-01    7.1e-03    1.7e-01    8.7e+02   7.0e-01
   8  -6.34e+02   9.6e-01    7.1e-03    1.7e-01    1.2e+03   1.0e+00
   9  -1.26e+03   9.8e-01    6.9e-03    1.6e-01    3.2e+03   1.7e+00
  10  -1.11e+03   9.2e-01    6.8e-03    1.6e-01    7.8e+02   2.2e+00
  11  -1.05e+03   8.7e-01    6.8e-03    1.6e-01    3.0e+02   2.7e+00
  12  -1.07e+03   8.5e-01    6.6e-03    1.6e-01    3.3e+02   3.1e+00
  13  -1.15e+03   8.0e-01    6.5e-03 

In [59]:
@benchmark mehrotraPCQuadBounds(SM3, 100, display = false)

BenchmarkTools.Trial: 
  memory estimate:  7.11 MiB
  allocs estimate:  21347
  --------------
  minimum time:     5.317 ms (0.00% GC)
  median time:      5.538 ms (0.00% GC)
  mean time:        5.990 ms (4.17% GC)
  maximum time:     11.300 ms (16.68% GC)
  --------------
  samples:          834
  evals/sample:     1

In [181]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path_pb, "\\SC50A.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4)
SM4 = SlackModel(QM4);

┌ Info: Using 'SC50A' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'MAXIM' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'CONST' as RHS (l. 153)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [182]:
res_mpc4 =  MehrotraPCQuadBounds(SM4, 20, tol_step_x = 1e-12);
display_results(res_mpc4)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |    -1.0000e+00   |         5.00e-01       |    1.30e+01  |    4.42e+00  |  0.00e+00  |  2.01e+01
   1 |    -8.1774e-01   |         5.84e+02       |    1.30e-02  |    3.06e+00  |  3.55e+01  |  1.45e+01
   2 |    -8.9022e-01   |         2.36e+01       |    1.30e-05  |    1.30e-01  |  7.66e-01  |  6.12e-01
   3 |    -9.3339e+00   |         3.42e+00       |    1.04e-05  |    1.23e-01  |  1.11e+02  |  5.93e-01
   4 |    -3.2552e+01   |         4.92e-01       |    7.80e-06  |    7.91e-02  |  3.01e+02  |  4.57e-01
   5 |    -5.4487e+01   |         1.03e-02       |    2.91e-06  |    6.01e-02  |  2.99e+02  |  3.24e-01
   6 |    -6.0759e+01   |         2.04e-02       |    8.31e-07  |    2.89e-02  |  1.03e+02  |  1.96e-01
   7 |    -6.3542e+01   |         1.94e-02       |    1.64e-07  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [1.76119e-11, 16.5687, 64.5751, 64.5751, 64.5751, 1.76119e-11…
  "lambda_opt" => [-4.28388e-15, -0.138705, -513941.0, -0.208058, -0.124759, -0…
  "s_l_opt"    => [0.013946, 1.49751e-14, 3.7416e-15, 3.74898e-15, 3.74898e-15,…
  "s_u_opt"    => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.02311…
  "n_iter"     => 12
  "pdd"        => 3.53456e-13
  "cond_rb"    => 9.53975e-14
  "cond_rc"    => 5.20628e-17

# Tests sur tous les problèmes

In [198]:
i = 0
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["BLEND.SIF"; "DFL001.SIF";  #QPSReader not working
                                                        "FORPLAN.SIF"; "GFRD-PNC.SIF"; 
                                                        "SIERRA.SIF"]) &&
                                        !(file_name in ["25FV47.SIF" ; "80BAU3B.SIF"; "BNL1.SIF"]) #singular exception
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        qpdata_i = readqps(pb_i) 
        SM_i = SlackModel(createQuadraticModel(qpdata_i))
        res_mpc_i =  MehrotraPCQuadBounds(SM_i, 20)
    end
    if i == 3
        break
        break
    end
end

ADLITTLE.SIF
Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     6.8001e+06   |         9.04e-01       |    8.83e+03  |    4.28e-01  |  0.00e+00  |  7.09e+03
   1 |     6.7988e+06   |         9.05e-01       |    8.83e+03  |    4.28e-01  |  1.75e+01  |  7.09e+03
   2 |     6.8101e+06   |         9.05e-01       |    8.83e+03  |    4.28e-01  |  1.89e+01  |  7.09e+03
   3 |     7.0361e+06   |         9.08e-01       |    8.83e+03  |    4.28e-01  |  1.09e+02  |  7.09e+03


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


   4 |     1.5516e+07   |         9.58e-01       |    8.83e+03  |    4.28e-01  |  3.66e+03  |  7.13e+03
   5 |     6.2212e+08   |         9.99e-01       |    8.83e+03  |    4.28e-01  |  2.59e+05  |  9.93e+03
   6 |     7.9439e+09   |         1.00e+00       |    8.83e+03  |    4.28e-01  |  3.13e+06  |  4.41e+04
   7 |     7.0007e+11   |         1.00e+00       |    8.83e+03  |    4.28e-01  |  2.96e+08  |  4.12e+06
   8 |     1.9253e+14   |         1.00e+00       |    8.83e+03  |    4.28e-01  |  8.20e+10  |  1.26e+09
   9 |     1.3514e+16   |         1.00e+00       |    8.83e+03  |    4.28e-01  |  5.69e+12  |  9.34e+10
  10 |     4.7848e+17   |         1.00e+00       |    8.82e+03  |    4.28e-01  |  1.99e+14  |  3.42e+12
  11 |     4.7848e+17   |         1.00e+00       |    8.82e+03  |    4.28e-01  |  5.88e+06  |  3.42e+12
  12 |     4.7848e+17   |         1.00e+00       |    8.82e+03  |    4.28e-01  |  5.13e+08  |  3.42e+12
  13 |     6.8985e+18   |         1.00e+00       |    8.82e+03  

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470



   3 |     3.6743e+01   |         3.61e+01       |    3.84e-16  |    7.04e-01  |  2.20e+02  |  3.31e+01
   4 |     6.7364e+01   |         1.07e+01       |    4.28e-16  |    3.31e-01  |  4.45e+01  |  1.76e+01
   5 |    -1.5940e+02   |         1.70e+00       |    5.84e-16  |    1.83e-01  |  3.13e+02  |  9.31e+00
   6 |    -4.0241e+02   |         3.17e-01       |    2.29e-15  |    4.03e-02  |  4.98e+02  |  3.99e+00
   7 |    -4.5934e+02   |         4.20e-02       |    1.03e-15  |    8.04e-03  |  1.06e+02  |  6.96e-01
   8 |    -4.6350e+02   |         5.62e-03       |    1.62e-15  |    2.43e-04  |  1.59e+01  |  6.09e-02
   9 |    -4.6475e+02   |         6.39e-06       |    9.55e-16  |    2.56e-07  |  2.11e+00  |  6.86e-05
  10 |    -4.6475e+02   |         6.39e-09       |    1.85e-15  |    2.56e-10  |  2.53e-03  |  6.86e-08
  11 |    -4.6475e+02   |         6.39e-12       |    2.19e-15  |    2.56e-13  |  2.57e-06  |  6.86e-11

 stopping criterion = ["pdd <= eps", "cond_rb <= eps", "cond_r

┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     2.0263e+03   |         1.00e+00       |    3.22e+06  |    8.22e-01  |  0.00e+00  |  7.40e+04
   1 |     2.0414e+03   |         1.94e+04       |    3.22e+06  |    8.22e-01  |  7.38e+00  |  7.40e+04

┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470



   2 |     1.7757e+03   |         2.09e+04       |    3.22e+06  |    8.22e-01  |  1.65e+02  |  7.40e+04
   3 |     1.6614e+03   |         2.26e+04       |    3.22e+06  |    8.22e-01  |  1.54e+02  |  7.40e+04
   4 |     1.3143e+03   |         2.69e+04       |    3.22e+06  |    8.22e-01  |  8.99e+01  |  7.40e+04
   5 |     1.2633e+03   |         3.01e+04       |    3.22e+06  |    8.22e-01  |  6.50e+01  |  7.40e+04
   6 |    -1.9401e+02   |         2.02e+05       |    3.22e+06  |    8.22e-01  |  1.91e+02  |  7.40e+04
   7 |    -1.5200e+03   |         2.53e+04       |    3.22e+06  |    8.22e-01  |  1.77e+02  |  7.40e+04
   8 |    -2.1502e+03   |         1.64e+04       |    3.22e+06  |    8.22e-01  |  1.54e+02  |  7.40e+04
   9 |    -3.0126e+03   |         1.19e+04       |    3.22e+06  |    8.22e-01  |  1.19e+02  |  7.40e+04
  10 |    -5.7926e+03   |         6.68e+03       |    3.22e+06  |    8.22e-01  |  3.00e+02  |  7.40e+04
  11 |    -8.6746e+03   |         4.12e+03       |    3.22e+06 

In [36]:
T = typeof(SM3.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1

In [37]:
typeof(SM3.model) == QuadraticModel

true

In [19]:
# obj  9.8723216072E+05
qpdata5 = readqps(string(path_pb, "\\25FV47.SIF"))
QM5 = createQuadraticModel(qpdata5)
SM5 = SlackModel(QM5);

┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [20]:
@show SM5

SM5 = SlackModel - Model with slack variables
  Problem name: Generic-slack
  Total variables………1876    Total constraints…821   
    free………………………………0         linear…………………………821   
    lower……………………………1571      nonlinear…………………0     
    upper……………………………305       equality……………………821   
    low/upp………………………0         lower……………………………0     
    fixed……………………………0         upper……………………………0     
    nnzh………………………………0         low/upp………………………0     
                              nnzj………………………………10705 
  Counters:
    obj……………0         grad…………0         cons…………0         jcon…………0         jgrad………0     
    jac……………0         jprod………0         jtprod……0         hess…………0         hprod………0     
    jhprod……0     


SlackModel - Model with slack variables
  Problem name: Generic-slack
  Total variables………1876    Total constraints…821   
    free………………………………0         linear…………………………821   
    lower……………………………1571      nonlinear…………………0     
    upper……………………………305       equality……………………821   
    low/upp………………………0         lower……………………………0     
    fixed……………………………0         upper……………………………0     
    nnzh………………………………0         low/upp………………………0     
                              nnzj………………………………10705 
  Counters:
    obj……………0         grad…………0         cons…………0         jcon…………0         jgrad………0     
    jac……………0         jprod………0         jtprod……0         hess…………0         hprod………0     
    jhprod……0     

In [23]:
res_mpc_5 =  MehrotraPCQuadBounds(SM5, 20)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     6.0144e+04   |         3.29e-01       |    5.78e+01  |    5.98e-01  |  0.00e+00  |  1.16e+02


SingularException: SingularException(0)